In [8]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
  
df = pd.read_csv('train.csv')
train=df
#train.insert(2,"clean",train['toxic']^1)
train_comments = train['comment_text']
train_toxic_comments=df.loc[train['toxic'] == 1]
df= pd.read_csv('test.csv')
test=df

test_comments=df['comment_text']
train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
df2 = pd.read_csv('turkce_yorumlar.csv')
df2.head()
df2.describe()
train2=df2
#train.insert(2,"clean",train['toxic']^1)
train_comments2 = train2['yorum']
y2= train2["toxic"]


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,SpatialDropout1D
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D,GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.models import Model

max_features = 100000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_comments) + list(test_comments))
train_tokenized = tokenizer.texts_to_sequences(train_comments)
test_tokenized = tokenizer.texts_to_sequences(test_comments)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
maxlen = 150
X_t = pad_sequences(train_tokenized, maxlen=maxlen)
X_te = pad_sequences(test_tokenized, maxlen=maxlen)
inp = Input(shape=(maxlen, ))
embed_size = 300

In [4]:
embeddings_index = {}
with open("glove.840B.300d.txt", encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [5]:
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.2)(x)
x1 = Bidirectional(LSTM(128, return_sequences=True))(x)
x2 = Bidirectional(LSTM(64, return_sequences=True))(x)
conc = concatenate([x1, x2])
#x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
#x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
#x=GlobalAveragePooling1D()(x)
avg_pool = GlobalAveragePooling1D()(conc)
max_pool = GlobalMaxPooling1D()(conc)
conc = concatenate([avg_pool, max_pool])
x = Dense(64, activation='relu')(conc)
x = Dropout(0.2)(x)
out = Dense(6, activation="sigmoid")(x)

#out = Dense(6, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("start fitting...")
model.fit(X_t,y, epochs=1, batch_size=32, verbose =1)

print("start predicting...")
y_pred = model.predict(X_te, batch_size=1024)

submission = pd.DataFrame.from_dict({'id': test['id']})
for idx, col in enumerate(list_classes):
    submission[col] = y_pred[:,idx]
submission.to_csv('submission_lstm_glove.csv', index=False)

start fitting...
Epoch 1/1
159571/159571 [==============================] - 5076s 32ms/step - loss: 0.0501 - acc: 0.9815
start predicting...


In [6]:

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    max_df=0.7,
    ngram_range=(1,1),
    )
word_vectorizer.fit(train_comments2)
X_t2=word_vectorizer.transform(train_comments2)

In [14]:
from sklearn.externals import joblib
clf = joblib.load('turkish_model.pkl') 

In [15]:
from tkinter import *
import tkinter as tk
from tkinter import scrolledtext
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

window = Tk()
window.title("Toxical Analysis")
window.geometry('1280x720')
#window.config(bg="#CCEEFF")
selected = IntVar()
rad1 = Radiobutton(window,text='Turkish', value=1, variable=selected)
rad1.pack(side=tk.RIGHT)
rad2 = Radiobutton(window,text='English', value=2, variable=selected)
rad2.pack(side=tk.RIGHT)

lbl = Label(window, text="    Enter Text For Toxical Analysis      ") 
lbl.pack()

txt = scrolledtext.ScrolledText(window,height=10, width=100)

txt.pack()

 



#background_image=tk.PhotoImage(file="background.png")
#background_label = tk.Label(window, image=background_image)
#background_label.place(x=0, y=0, relwidth=1, relheight=1)
#figure1 = Figure(figsize=(5,4), dpi=100) 
#subplot1 = figure1.add_subplot(111) 

def clicked(): 
    
    test=txt.get("1.0","end-1c")
    df=pd.DataFrame([str(test)],columns=['comment'])
    comment=df['comment']
    
    if(selected.get()==1):
        vect=word_vectorizer.transform(comment)
        #p=clf.predict_proba(vect)[0][1]
        p=clf.predict(vect)
        result="Toxic= "+str(p)
    elif(selected.get()==2):
        vect=tokenizer.texts_to_sequences(comment)
        vect=pad_sequences(vect, maxlen=maxlen)
        #print(vect)

        predictions=loaded_model.predict(vect)
        result="\n\nToxic= "+str(predictions[0][0])+"\nSevere Toxic= "+str(predictions[0][1])
        result+="\nObscene= "+str(predictions[0][2])+"\nThreat= "+str(predictions[0][3])+"\nInsult= "+str(predictions[0][4])
        result+="\nIdentity Hate= "+str(predictions[0][5])
    else:
        result="Please Select A Language"
    
    

    
    
    #objects = ('Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate')
    #y_pos = np.arange(len(objects))
    #subplot1.bar(objects,predictions[0], color = 'g') 
    #bar1 = FigureCanvasTkAgg(figure1, window) 
    #bar1.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=0)
    #print(predictions[0])
    
    lbl2.configure(text=result)
    
    
def clicked2(): 
    
    
    
    if(selected.get()==1):
        random_text=df2.sample()["yorum"].values[0]
        print(random_text)
        txt.delete('1.0', END)
        txt.insert(INSERT, random_text)
    elif(selected.get()==2):
        random_text=df.sample()["comment_text"].values[0]
        print(random_text)
        txt.delete('1.0', END)
        txt.insert(INSERT, random_text)
    else:
        lbl2.configure(text="Please Select A Language")

btn = Button(window, text="Start The Analysis", command=clicked,background = 'black', foreground = "white")
btn.pack()
btn2 = Button(window, text="Add Random Comment", command=clicked2,background = 'black', foreground = "white")
btn2.pack()
lbl2 = Label(window, text="*",justify=tk.LEFT) 
lbl2.pack(side=tk.LEFT)
window.mainloop()

Bütünüyle eksiksiz ve harika
Eğlenceli zaman geçirmek için iyi bir yöntem
Bu oyunu oynamadan ölmeyin!
İndirimde bu oyunu kaçırmayın kaçırtmayın
oyun bence çok güzel fakat online arkadaşlarınla oynana bilseydi dahada güzel olurdu
Ne kadar oynasanız da sıkmayan bir oyun
oyun çok eğlenceli kesinlikle tavsiye ediyorum
BEN NEDEN PARA VERDIM OROSPU ÇOCUKLARI
The photos seem to suggest one point of view (that Asbury Park's a piece of crap [pardon the French]) and the article suggests another (Asbury Park: Gettin' better every day).  Thoughts?
== Leona Lews... == 

 ...is a slag. 

 I Abi Branning frankly since she's pure and innocent whereas Leona is a whore. Anuway I want to cuddle up with Lorna Fitzgerald as she's so sweet and cute and magical.


In [29]:
#!jupyter nbconvert --to script config_template.ipynb

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

classifier = LogisticRegression(class_weight='balanced')
classifier.fit(X_t2,y2)
joblib.dump(classifier, 'turkish_model.pkl') 

['turkish_model.pkl']

In [13]:
clf = joblib.load('turkish_model.pkl') 